In [1]:
import numpy as np
import pandas as pd

In [9]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import re
from PIL import Image
import requests
import random
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

In [2]:
books = pd.read_csv("Books.csv", low_memory=False)
ratings = pd.read_csv("Ratings.csv", low_memory=False)
users = pd.read_csv("Users.csv", low_memory=False)
print("books : ")
print(books.head(3))
print("ratings : ")
print(ratings.head(3))
print("users : ")
print(users.head(3))

books : 
         ISBN            Book-Title           Book-Author Year-Of-Publication  \
0  0195153448   Classical Mythology    Mark P. O. Morford                2002   
1  0002005018          Clara Callan  Richard Bruce Wright                2001   
2  0060973129  Decision in Normandy          Carlo D'Este                1991   

                 Publisher                                        Image-URL-S  \
0  Oxford University Press  http://images.amazon.com/images/P/0195153448.0...   
1    HarperFlamingo Canada  http://images.amazon.com/images/P/0002005018.0...   
2          HarperPerennial  http://images.amazon.com/images/P/0060973129.0...   

                                         Image-URL-M  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0060973129.0...   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/0195153448.0...  
1  htt

In [3]:
books_data=books.merge(ratings,on="ISBN")
books_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [10]:
df=books_data.copy()
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.drop(columns=["ISBN","Year-Of-Publication","Image-URL-S","Image-URL-M"],axis=1,inplace=True)
df.drop(index=df[df["Book-Rating"]==0].index,inplace=True)
df["Book-Title"]=df["Book-Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())
df.head()

,Book-Title,Book-Author,Publisher,Image-URL-L,User-ID,Book-Rating
1,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,8,5
3,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,11676,8
5,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,67544,8
8,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,116866,9
9,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,123629,9


In [11]:
# 책 평가 200번 초과한 사람들 -> new_df
new_df=df[df['User-ID'].map(df['User-ID'].value_counts()) > 200]  # Drop users who vote less than 200 times.
users_pivot=new_df.pivot_table(index=["User-ID"],columns=["Book-Title"],values="Book-Rating")
users_pivot.fillna(0,inplace=True)


In [12]:
# 유저의 최고 평점 책 5권 골라 리턴
def users_choice(id):
    
    users_fav=new_df[new_df["User-ID"]==id].sort_values(["Book-Rating"],ascending=False)[0:5]
    return users_fav

In [13]:
# cosine similarity 활용해 가장 유사한 유저 5명 찾아 user id 리턴
def user_based(new_df,id):
    if id not in new_df["User-ID"].values:
        print("❌ User NOT FOUND ❌")
        
        
    else:
        index=np.where(users_pivot.index==id)[0][0]
        similarity=cosine_similarity(users_pivot)
        similar_users=list(enumerate(similarity[index]))
        similar_users = sorted(similar_users,key = lambda x:x[1],reverse=True)[0:5]
    
        user_rec=[]
    
        for i in similar_users:
                data=df[df["User-ID"]==users_pivot.index[i[0]]]
                user_rec.extend(list(data.drop_duplicates("User-ID")["User-ID"].values))
        
    return user_rec

In [14]:
# user(userid와 유사한 유저 목록) 와 user_id가 평가한 책들에서 
# userid가 아직 읽지 않은 책 상위 5권 리턴
def common(new_df,user,user_id):
    x=new_df[new_df["User-ID"]==user_id]
    recommend_books=[]
    user=list(user)
    for i in user:
        y=new_df[(new_df["User-ID"]==i)]
        books=y.loc[~y["Book-Title"].isin(x["Book-Title"]),:]
        books=books.sort_values(["Book-Rating"],ascending=False)[0:5]
        recommend_books.extend(books["Book-Title"].values)
        
    return recommend_books[0:5]

In [34]:
user_id = random.choice(new_df["User-ID"].values)
user_choice_df = pd.DataFrame(users_choice(user_id))
user_favorite = users_choice(user_id)
n = len(user_choice_df["Book-Title"].values)
print("🟦 USER: {} ".format(user_id))

# YOUR FAVORITE BOOKS
user_favorite_books = []
for i in range(n):
    book_title = user_choice_df["Book-Title"].tolist()[i]
    rating = round(new_df[new_df["Book-Title"] == book_title]["Book-Rating"].mean(), 1)
    user_favorite_books.append((book_title, rating))

# YOU MAY ALSO LIKE THESE BOOKS
user_based_rec = user_based(new_df, user_id)
books_for_user = common(new_df, user_based_rec, user_id)
recommended_books = []
for i in range(5):
    book_title = books_for_user[i]
    rating = round(new_df[new_df["Book-Title"] == book_title]["Book-Rating"].mean(), 1)
    recommended_books.append((book_title, rating))

# 출력
print("YOUR FAVORITE BOOKS:")
for book, rating in user_favorite_books:
    print(f"Book: {book}, Rating: {rating}")
print("--------------------------------------------------------------------------------------")
print("Recommend these Books!!")
for book, rating in recommended_books:
    print(f"Book: {book}, Rating: {rating}")    

🟦 USER: 23902 
YOUR FAVORITE BOOKS:
Book: Last Train to Memphis The Rise of Elvis Presley, Rating: 10.0
Book: The Best of Oscar Wilde Selected Plays and Writings, Rating: 10.0
Book: Brideshead Revisited, Rating: 10.0
Book: Doctor Zhivago, Rating: 10.0
Book: Complete Works of William Shakespeare, Rating: 10.0
--------------------------------------------------------------------------------------
Recommend these Books!!
Book: Down in the Garden, Rating: 10.0
Book: Private Wars, Rating: 10.0
Book: Traveling Light Releasing the Burdens You Were Never Intended to Bear, Rating: 10.0
Book: Where the Broken Heart Still Beats The Story of Cynthia Ann Parker, Rating: 10.0
Book: Get Off the Unicorn, Rating: 8.6
